In [5]:
import json
class obj:
    # constructor
    def __init__(self, dict1):
        self.__dict__.update(dict1)
args = {
    "candidates": 20,
    "device": 'mps',
    "dev_dir":'datasets/development/processed_dev',
    "train_dir":'datasets/development/processed_dev',
    "dictionary_path": 'datasets/development/dev_dictionary.txt',
    "max_length": 25,
    "model_name_or_path": 'dmis-lab/biobert-base-cased-v1.1',
    "batch_size": 16,
    "epochs": 1,
    "loss_fn": "nll",
    "contextualized": False,
    "similarity_type": 'binary'
}
args = json.loads(json.dumps(args), object_hook=obj)
vars(args)

from importlib import reload

import numpy as np
import time
import torch
from tqdm import tqdm
from transformers import (
    AutoModel,
    AutoTokenizer
)

# Local modules
from src.candidateDataset import CandidateDataset
from src.rerankNet import RerankNet
from src.umls import Umls
import src.utils as utils

In [ ]:
LOGGER = utils.init_logging()
LOGGER.info(args)
utils.init_seed(42)
bert = AutoModel.from_pretrained(args.model_name_or_path).to(args.device)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

# Set loss function
if args.loss_fn=='nll':
    loss_fn = utils.marginal_nll
elif args.loss_fn=='bce':
    loss_fn = utils.marginal_nll
else:
    raise Exception(f"Invalid loss function {args.loss_fn}")
    
# Build model
model = RerankNet(bert, device = args.device)

# Load UMLS data
umls = Umls('umls/processed')
LOGGER.info("UMLS data loaded")

# Load dictionary
dictionary = utils.load_dictionary(args.dictionary_path)
LOGGER.info("Dictionary loaded")

# Load training data
train_mentions = utils.load_mentions(args.train_dir)
train_set = CandidateDataset(train_mentions, dictionary, tokenizer, args.max_length, args.candidates, args.similarity_type, umls) 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=args.batch_size, shuffle=True)

# Load dev data for validation
dev_mentions = utils.load_mentions(args.dev_dir)
LOGGER.info("Mentions loaded")

In [8]:
# Training loop
for epoch in range(args.epochs):
        ############## Candidate Generation ##############
        train_candidate_idxs = utils.get_topk_candidates(
                dict_names=list(dictionary[:,0]), 
                mentions=train_mentions, 
                tokenizer=tokenizer, 
                encoder=bert, 
                max_length=args.max_length, 
                device=args.device, 
                topk=args.candidates)
                                
        # Add candidates to training dataset
        train_set.set_candidate_idxs(train_candidate_idxs)

        ###################### Train ######################
        # Train encoder to properly rank candidates
        train_loss = 0
        train_steps = 0
        model.train()
        for i, data in tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Training epoch {epoch}'):
                model.optimizer.zero_grad()
                batch_x, batch_y = data
                batch_pred = model(batch_x)
                loss = loss_fn(batch_pred, batch_y.to(args.device))
                # loss.backward() #TODO: Not working on the mac
                model.optimizer.step()
                train_loss += loss.item()
                train_steps += 1

        train_loss = train_loss / (train_steps + 1e-9)
        LOGGER.info('Epoch {}: loss/train_per_epoch={}/{}'.format(epoch,train_loss,epoch))

        #################### Evaluate ####################
        # Get candidates on dev dataset
        dev_candidate_idxs = utils.get_topk_candidates(
                dict_names=list(dictionary[:,0]), 
                mentions=dev_mentions, 
                tokenizer=tokenizer, 
                encoder=bert, 
                max_length=args.max_length, 
                device=args.device, 
                topk=5) # Only need top five candidates to evaluate performance

        # Log performance on dev after each epoch
        results = utils.evaluate(dev_mentions, dictionary[dev_candidate_idxs], umls)
        if 'acc1' in results: LOGGER.info("Epoch {}: acc@1={}".format(epoch,results['acc1']))
        if 'acc5' in results: LOGGER.info("Epoch {}: acc@5={}".format(epoch,results['acc5']))
        if 'umls_similarity' in results: LOGGER.info("Epoch {}: umls_similarity={}".format(epoch,results['umls_similarity']))

Training epoch 0:   0%|          | 0/1 [00:02<?, ?it/s]


NotImplementedError: The operator 'aten::index.Tensor' is not current implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [397]:
import src.utils as utils
reload(utils)

/Users/evan/code/thesis/src/utils.py:79: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  # Check all annotations were fixed


<module 'src.utils' from '/Users/evan/code/thesis/src/utils.py'>

In [64]:
print(batch_pred.shape, batch_y.shape)
expected_ixs = utils.retrieve_candidates(batch_y, topk=5)
expected = torch.stack([batch_y[i][expected_ixs[i]] for i in range(batch_pred.shape[0])])
expected

torch.Size([8, 20]) torch.Size([8, 20])


tensor([[1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [54]:
predicted_ixs = utils.retrieve_candidates(batch_pred.detach().cpu(), topk=5)
predicted = torch.stack([batch_y[i][predicted_ixs[i]] for i in range(batch_pred.shape[0])])
predicted

tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [55]:
torch.nn.functional.binary_cross_entropy(predicted, expected)

tensor(30.)

In [57]:
input = torch.randn(3, 2, requires_grad=True)
target = torch.rand(3, 2, requires_grad=False)
loss = torch.nn.functional.binary_cross_entropy(torch.sigmoid(input), target)
loss

tensor(0.8716, grad_fn=<BinaryCrossEntropyBackward0>)

In [62]:
input, torch.sigmoid(input), target

(tensor([[ 0.1844, -0.1064],
         [ 0.1777,  1.3448],
         [-0.6941, -1.8600]], requires_grad=True),
 tensor([[0.5460, 0.4734],
         [0.5443, 0.7933],
         [0.3331, 0.1347]], grad_fn=<SigmoidBackward0>),
 tensor([[0.7057, 0.1659],
         [0.3779, 0.1089],
         [0.0501, 0.6333]]))